
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [99]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

import numpy as np
from sklearn.model_selection import train_test_split
import csv

with open('data/Corona_NLP_train.csv', 'r', encoding='utf-8-sig', errors = 'replace') as file:
    reader = csv.DictReader(file)
    rows = [line for line in reader if line["Sentiment"] != "Neutral"]

X = np.array([row["OriginalTweet"] for row in rows], dtype = 'str')
Y = np.array([row["Sentiment"] for row in rows], dtype = str)
Y = np.where((Y == "Positive") | (Y == "Extremely Positive"), 1, 0)

print(X.shape)
print(Y.shape)

X_train, x_intermediary, Y_train, y_intermediary = train_test_split(X, Y, test_size = 0.4, random_state = 42)
X_valid, X_test, Y_valid, Y_test = train_test_split (x_intermediary, y_intermediary, test_size = 0.375, random_state = 42)

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)
print(X_test.shape, Y_test.shape)

(33444,)
(33444,)
(20066,) (20066,)
(8361,) (8361,)
(5017,) (5017,)


In [116]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer()

model = Pipeline([
                ('vectorizer', CountVectorizer()),
                ('lr', LogisticRegression(solver="liblinear", max_iter=1000))
            ])

model.fit(X_train, Y_train)

,steps,"[('vectorizer', ...), ('lr', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [132]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

from sklearn.metrics import multilabel_confusion_matrix

prediction = model.predict(X_test)

mcm = multilabel_confusion_matrix(Y_test, prediction)
print(mcm)

tn = mcm[:, 0, 0]
tp = mcm[:, 1, 1]
fn = mcm[:, 1, 0]
fp = mcm[:, 0, 1]

precision = tp / (tp + fp)
recall = tp / (tp + fn)

precision_0 = precision[0]
precision_1 = precision[1]
recall_0 = recall[0]
recall_1 = recall[1]

print(precision_0, precision_1, recall_0, recall_1)

[[[2375  288]
  [ 349 2005]]

 [[2005  349]
  [ 288 2375]]]
0.8744003488879197 0.8718795888399412 0.8517417162276976 0.8918512955313556


In [180]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost

    predictions = []
    total_cost = 0

    probabilities = model.predict_proba(X)

    for prob in probabilities:
        if prob[1] >= threshold:
            predictions.append(1)
        else:
            predictions.append(0)

    for pred, true in zip(predictions, Y):
        if pred == true:
            total_cost += 0
        elif pred == 0 and true == 1:
            total_cost +=1
        else:
            total_cost += 5

    average_cost = total_cost / len(Y)
            
    return average_cost

In [186]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

best_threshold = 0
best_cost = float("inf")

for i in range(0,11, 1):
    threshold = i/10
    avg_cost = cost(model, threshold, X_test, Y_test)

    if avg_cost < best_cost:
        best_cost = avg_cost
        best_threshold = threshold

print(best_threshold, best_cost)

optimal_threshold = best_threshold
cost_at_optimal_threshold = best_cost

0.8 0.282240382698824


In [218]:

# Part 6

t = np.sqrt(np.log(0.01/2)/(-2*len(Y_valid)))

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
cost_interval_valid = (cost_at_optimal_threshold_valid - t, cost_at_optimal_threshold_valid + t)

print(cost_at_optimal_threshold_valid)
print(cost_interval_valid)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

0.2687477574452817
(np.float64(0.2509475577388445), np.float64(0.2865479571517189))


In [216]:

# Part 7

from scipy.optimize import bisect

probabilities = model.predict_proba(X_valid)

predictions = []

for prob in probabilities:
        if prob[1] >= threshold:
            predictions.append(1)
        else:
            predictions.append(0)
            
costs = []
for pred, true in zip(predictions, Y_valid):
    cost_per_tweet = 0
    if pred == true:
        cost_per_tweet += 0
    elif pred == 0 and true ==1:
        cost_per_tweet += 1
    else:
        cost_per_tweet += 5
    costs.append(cost_per_tweet)

costs = np.array(costs)

variance_of_C = np.mean(costs**2) - np.mean(costs)**2  # V(C) = E[C^2] - (E[C])^2

a = 5
delta = 0.01
n = len(Y_valid)
c = -(a**2 / (len(Y_valid) * variance_of_C)) * np.log(0.01)

def h(u):
    return (1 + u) * np.log(1 + u) - u

u = bisect(lambda x: h(x) - c, 0, 100)

epsilon = (variance_of_C * u) / a

interval_of_C = (cost_at_optimal_threshold_valid - epsilon, cost_at_optimal_threshold_valid + epsilon)

print(variance_of_C)
print(interval_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

0.24787415146621722
(np.float64(0.25132889329642205), np.float64(0.2861666215941413))
